# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
llm_model = os.environ['LLM_MODEL'] 

In [3]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [4]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [5]:
from langchain.indexes import VectorstoreIndexCreator

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

embeddings = OpenAIEmbeddings()
llm = OpenAI(temperature=0)


In [7]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
       embedding=embeddings
    
).from_loaders([loader])

/media/mark/data/ES/building_ai_applications-labs/Langchain/myenv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [9]:
response = index.query(query, llm=llm) 


In [10]:
display(Markdown(response))



| Name | Description | Sun Protection Rating |
| --- | --- | --- |
| Men's Tropical Plaid Short-Sleeve Shirt | Made of 100% polyester, UPF 50+ rated, wrinkle-resistant, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's Plaid Tropic Shirt, Short-Sleeve | Made of 52% polyester and 48% nylon, UPF 50+ rated, SunSmart technology blocks 98% of harmful UV rays, wrinkle-free, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Men's TropicVibe Shirt, Short-Sleeve | Made of 71% nylon and 29% polyester, UPF 50+ rated, front and back cape venting, two front bellows pockets, imported | SPF 50+, blocks 98% of harmful UV rays |
| Sun Shield Shirt | Made of 78% nylon and 22% Lycra Xtra Life fiber, UPF 50+ rated, wicks moisture, fits comfortably over swimsuit, abrasion resistant

## Step By Step

In [11]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path=file)

In [12]:
docs = loader.load()

In [13]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [14]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

/tmp/ipykernel_43436/2047715053.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [15]:
embed = embeddings.embed_query("Hi my name is Harrison")

In [16]:
print(len(embed))

1536


In [17]:
print(embed[:5])

[-0.02196465528695117, 0.006758838256223806, -0.018249490165056663, -0.03923515029463157, -0.014007174091135742]


In [18]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [19]:
query = "Please suggest a shirt with sunblocking"

In [20]:
docs = db.similarity_search(query)

In [21]:
len(docs)

4

In [22]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [23]:
retriever = db.as_retriever()

In [24]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)

In [25]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])


In [26]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


/tmp/ipykernel_43436/2552755760.py:1: LangChainDeprecationWarning: The method `BaseChatModel.call_as_llm` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.call_as_llm(f"{qdocs} Question: Please list all your \


In [27]:
display(Markdown(response))

Certainly! Below is a table summarizing the shirts with sun protection:

| Name                               | Description                                                                                                           | Size & Fit                  | Fabric & Care                                                                 | Additional Features                                                                 |
|------------------------------------|-----------------------------------------------------------------------------------------------------------------------|-----------------------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------------|
| Sun Shield Shirt                   | High-performance sun shirt with UPF 50+ protection, slightly fitted, falls at hip.                                     | Slightly Fitted             | 78% nylon, 22% Lycra Xtra Life fiber. Handwash, line dry.                     | Moisture-wicking, abrasion-resistant, fits over swimsuit, imported.                  |
| Men's Plaid Tropic Shirt           | Ultracomfortable short-sleeve shirt with UPF 50+, originally designed for fishing, great for travel.                   | Not specified               | 52% polyester, 48% nylon. Machine washable and dryable.                       | Wrinkle-free, quick-drying, front and back cape venting, two front bellows pockets.  |
| Men's TropicVibe Shirt             | Lightweight short-sleeve shirt with UPF 50+, relaxed fit, ideal for hot weather.                                       | Traditional Fit             | Shell: 71% Nylon, 29% Polyester. Lining: 100% Polyester knit mesh. Machine wash and dry. | Wrinkle-resistant, front and back cape venting, two front bellows pockets, imported. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightest hot-weather shirt with UPF 50+, traditional fit, relaxed through chest, sleeve, and waist.                     | Traditional Fit             | 100% polyester. Wrinkle-resistant.                                            | Front and back cape venting, two front bellows pockets, imported.                    |

Each shirt offers UPF 50+ sun protection, blocking 98% of the sun's harmful rays, and is designed for comfort and functionality in hot weather.

In [28]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [29]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [30]:
response = qa_stuff.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [31]:
display(Markdown(response["result"]))

Here is a table summarizing the shirts with sun protection:

| Name                                      | Summary                                                                                           |
|-------------------------------------------|---------------------------------------------------------------------------------------------------|
| Men's Tropical Plaid Short-Sleeve Shirt   | A lightweight, wrinkle-resistant shirt made of 100% polyester, offering UPF 50+ sun protection. It features front and back cape venting and two front bellows pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve    | Designed for fishing and travel, this shirt is made of 52% polyester and 48% nylon. It provides UPF 50+ protection, is wrinkle-free, and quickly evaporates perspiration. |
| Men's TropicVibe Shirt, Short-Sleeve      | This shirt combines 71% nylon and 29% polyester with a 100% polyester knit mesh lining. It offers UPF 50+ protection, is wrinkle-resistant, and includes front and back cape venting. |
| Sun Shield Shirt                          | Made of 78% nylon and 22% Lycra Xtra Life fiber, this shirt is slightly fitted, wicks moisture, and is abrasion-resistant. It provides UPF 50+ protection and is recommended by The Skin Cancer Foundation. |

Each shirt provides the highest rated sun protection possible, blocking 98% of the sun's harmful rays.